<a href="https://colab.research.google.com/github/claredavies/ImageIndexing/blob/master/Image_Indexing_Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 11762 Content-Based Image Retrieval
## Master's Degree in Intelligent Systems
### University of the Balearic Islands

---

**Before you turn this problem in, please put your full names and DNIs (or NIEs) below, and execute the cell:**

In [1]:
NAME  = "Clare Davies"
DNI   = "99999999R"

In [4]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [8]:
%cd drive/MyDrive/ImageIndexing

/content/drive/MyDrive/ImageIndexing


Make sure you fill in any place that says `YOUR CODE HERE` or `YOUR ANSWER HERE`. **Justify** all of your answers, **graphically** wherever possible. Remember that this notebook will be considered as a report to the work done during the assignment.

---

In [12]:
!unzip iric_dev_kit.zip

Archive:  iric_dev_kit.zip
   creating: iric_dev_kit/
  inflating: iric_dev_kit/iric.yml   
  inflating: iric_dev_kit/README     
   creating: iric_dev_kit/vocabs/
  inflating: iric_dev_kit/vocabs/sift_16c.npy  
  inflating: iric_dev_kit/vocabs/sift_64c.npy  
  inflating: iric_dev_kit/vocabs/sift_32c.npy  
  inflating: iric_dev_kit/vocabs/surf_l2_16c.arff  
  inflating: iric_dev_kit/vocabs/surf_l2_128c.arff  
  inflating: iric_dev_kit/vocabs/surf_l2_64c.arff  
  inflating: iric_dev_kit/vocabs/surf_l2_32c.arff  
   creating: iric_dev_kit/iric_utils/
  inflating: iric_dev_kit/iric_utils/read_descriptors.py  
 extracting: iric_dev_kit/iric_utils/__init.py__  
  inflating: iric_dev_kit/iric_utils/eval_holidays.py  


In [13]:
# Setup code for this assignment
import cv2
import math
import numpy as np
import os
import scipy.cluster.vq as vq
import tqdm
import zipfile

## Adding parent folder to find other libs
import sys
if ".." not in sys.path:
    sys.path.insert(0,"..")
    
import iric_dev_kit.iric_utils.eval_holidays as ev
import iric_dev_kit.iric_utils.read_descriptors as rd

# Configuring Matplotlib
from matplotlib import pyplot as plt
%matplotlib inline

## Introduction
In this assignment, you will implement and evaluate different methods for indexing images. As usual during this course, we will use the [INRIA Holidays](http://lear.inrialpes.fr/people/jegou/data.php) dataset. **Check the Assignment 1 to further information about this dataset.**

We also need the provided script to evaluate a CBIR system on this dataset. Remember that the performance is measured computing the **mean average precision** (mAP) over all queries. **Check also the Assignment 1 to remember how to use this script and the different functions it offers.**

### Loading images
As we did in Assignment 1, for managing images, we will create four lists:
- **`query_names`**: File names of the *query* images
- **`query_imgs`**: *Query* images loaded using OpenCV2
- **`train_names`**: File names of the *train* (database) images
- **`train_imgs`**: *Train* images loaded using OpenCV2

In this assignment, we will use the original holidays dataset:

In [14]:
!unzip holidays_mini.zip

Archive:  holidays_mini.zip
   creating: holidays_mini/
  inflating: holidays_mini/holidays_images.dat  
   creating: holidays_mini/images/
  inflating: holidays_mini/images/100000.jpg  
  inflating: holidays_mini/images/100001.jpg  
  inflating: holidays_mini/images/100002.jpg  
  inflating: holidays_mini/images/100100.jpg  
  inflating: holidays_mini/images/100101.jpg  
  inflating: holidays_mini/images/100200.jpg  
  inflating: holidays_mini/images/100201.jpg  
  inflating: holidays_mini/images/100300.jpg  
  inflating: holidays_mini/images/100301.jpg  
  inflating: holidays_mini/images/100302.jpg  
  inflating: holidays_mini/images/100400.jpg  
  inflating: holidays_mini/images/100401.jpg  
  inflating: holidays_mini/images/100500.jpg  
  inflating: holidays_mini/images/100501.jpg  
  inflating: holidays_mini/images/100502.jpg  
  inflating: holidays_mini/images/100503.jpg  
  inflating: holidays_mini/images/100600.jpg  
  inflating: holidays_mini/images/100601.jpg  
  inflating: h

In [15]:
# Separating the dataset into query and train images
query_names = []
query_imgs = []
train_names = []
train_imgs = []

# with open('../holidays/holidays_images.dat') as f:

with open('holidays_mini/holidays_images.dat') as f:
    for line in f:
        imname = line.strip()
        imno = int(imname[:-len(".jpg")])
        # img = cv2.imread('../holidays/images/' + imname)

        img = cv2.imread('holidays_mini/images/' + imname)
        # Resize the images for a faster operation in this assignment
        img = cv2.resize(img, None, fx=0.25, fy=0.25, interpolation = cv2.INTER_CUBIC)
    
        # Checking if this is a query image
        if imno % 100 == 0:
            query_names.append(imname)
            query_imgs.append(img)
        else:
            train_names.append(imname)
            train_imgs.append(img)

print(len(query_names))
print(len(train_names))

19
31


## Loading SIFT descriptors
In this assignment we will create four additional lists:
- **`query_kps`**: A list of lists of keypoints (cv2.KeyPoint) extracted from the *query* images
- **`query_desc`**: A list of numpy arrays including, for each set of keypoints, the SIFT descriptors extracted from the *query* images
- **`train_kps`**: A list of lists of keypoints (cv2.KeyPoint) extracted from the *train* (database) images
- **`train_desc`**: A list of numpy arrays including, for each set of keypoints, the SIFT descriptors extracted from the *train* images

Unlike in Assigment 1, now you will be provided with a set of SIFT descriptors for each image, and, therefore, you do not need to create these lists from scratch. First, download the descriptors from [here](https://uibes-my.sharepoint.com/:u:/g/personal/egf350_id_uib_es/Eam8Ld8YDaJAhNr91YVdAZIB_wVZJ8kzzKD7BR6R3LziMw).

> **Unzip this file into the root directory of the development kit, at the same level of the datasets.**

Now, a new directory called `siftgeo` should be in your workspace, containing the set of SIFT descriptors for each image of the dataset. These descriptors are stored in binary format and, thus, you are also provided with some tools to load them. To be more precise, you can call the function `load_SIFT_descriptors` to load the descriptors of a list of images:

In [5]:
import tarfile
def unzip_tarfile(filename):
  try:
    tar = tarfile.open(filename)
    tar.extractall()
    tar.close()
  except EOFError:
    print("error")

In [17]:
unzip_tarfile("siftgeo.tar.gz")

In [19]:
%cd iric_dev_kit

/content/drive/MyDrive/ImageIndexing/iric_dev_kit


In [20]:
# Loading descriptors
query_kps, query_desc = rd.load_SIFT_descriptors(query_names, max_desc=1000)
train_kps, train_desc = rd.load_SIFT_descriptors(train_names, max_desc=1000)

# Some prints
print(len(query_kps))
print(len(train_kps))
print(len(query_desc))
print(len(train_desc))
print(query_desc[0].shape)
print(query_desc[0])

19
31
19
31
(1000, 128)
[[10.  6. 52. ... 15.  4.  0.]
 [16. 50. 12. ... 15.  4.  0.]
 [10. 11. 58. ...  7.  4.  4.]
 ...
 [27. 15.  0. ... 16.  8. 12.]
 [51. 47. 14. ... 35. 26.  0.]
 [ 2. 37. 25. ... 47. 13.  8.]]


For development purposes, we use the parameter `max_desc` to load a maximum number (1000) of the descriptors. This will speed up the execution of the rest of the notebook, while the decrease in performance will be minimum.

> **Some images do not have keypoints/descriptors. Take this into account when you develop your solution.**

## $k$-d trees and LSH 
Let's start coding. At this section, you will develop a retrieval system using $k$-d trees and Locality Sensitive Hashing (LSH). 

### General framework
As we did in the first assignment, you first will develop some utilities to simplify your work. Write a function called `search_image` to search an image in a generic index (database). You should search each descriptor of the given query image and obtain its two closest SIFT descriptors in the database. Next, the initial set of matches should be filtered using the **NNDR criterion (use 0.8 as ratio)**, as you did in the previous assignment. For each database image, its final score with regard to this query image will be the **number of correct matches** with this image:

In [21]:
def search_image(descs, index, id_to_name):
    """
    Search an image in the index
    
    - descs: A numpy array. This is the set descriptors extracted from the query image
    - index: OpenCV FLANN index to search for descriptors.
    - id_to_name: An associative list to link every image index to its real name
        e.g. id_to_name[0] = '100001.jpg', id_to_name[1] = '100002.jpg'
  
    RETURN: 
    - An ordered list of similar images, e.g.: ['100101.jpg', '100202.jpg', ...]
    """

    # Query the index for the closest descriptors
    _, indices = index.knnSearch(descs, k=2, params={})
    
    # Filter matches using NNDR criterion
    threshold = 0.8
    mask = np.where(_[:, 0] < threshold * _[:, 1])
    indices = indices[mask][:, 0]
    
    # Count the number of matches for each image
    counts = np.zeros(len(id_to_name), dtype=np.int32)
    for index in indices:
        counts[index] += 1
        
    # Sort images by number of matches
    order = np.argsort(-counts)
    result = [id_to_name[i] for i in order if counts[i] > 0]
    
    return result

Now, write a function called `compute_mAP`. Given a list of query images and a trained index, this function should return a Python dictionary with the ordered results for each query along with the computed mAP:

In [22]:
print(query_names)

['100000.jpg', '100100.jpg', '100200.jpg', '100300.jpg', '100400.jpg', '100500.jpg', '100600.jpg', '100700.jpg', '100800.jpg', '100900.jpg', '101000.jpg', '101100.jpg', '101200.jpg', '101300.jpg', '101400.jpg', '101500.jpg', '101600.jpg', '101700.jpg', '101800.jpg']


In [23]:
print(query_desc[0])

[[10.  6. 52. ... 15.  4.  0.]
 [16. 50. 12. ... 15.  4.  0.]
 [10. 11. 58. ...  7.  4.  4.]
 ...
 [27. 15.  0. ... 16.  8. 12.]
 [51. 47. 14. ... 35. 26.  0.]
 [ 2. 37. 25. ... 47. 13.  8.]]


In [24]:
def compute_mAP(query_names, query_desc, index, id_to_name):
    """
    Perform a search for a list of query images against the database.
    
    - query_names: An ordered list with the names of the query images
    - query_desc: A list containing numpy arrays of size (ndesc_for_this_image, 128)
                  Each numpy array i corresponds to the descriptors found at image i
    - index: FLANN index
    - id_to_name: An associative array to link every image index to its real name
                  e.g. id_to_name[0] = '100001.jpg', id_to_name[1] = '100002.jpg'
  
    RETURN: 
    - total_results: A dictionary containing, for each query image, an sorted list of the database images
    - m_ap: Mean Average Precision averaged over all queries
    """
    total_results = {}
    m_ap = 0.0

    flann = pyflann.FLANN()

    # Perform nearest neighbor search for each query image
    for i, (name, desc) in enumerate(zip(query_names, query_desc)):
      search_result =  search_image(desc, index, id_to_name)

    # YOUR CODE HERE

    #  # Initialize variables
    # total_results = {}
    # sum_ap = 0
    # n_queries = len(query_names)
    
    # # Compute results for each query image
    # for i in range(n_queries):
    #     query_name = query_names[i]
    #     query_descs = query_desc[i]
        
    #     # Perform the search
    #     db_names = search_image(query_descs, index, id_to_name)
    #     total_results[query_name] = db_names
        
    #     # Compute AP using the available function
    #     ap = compute_AP(query_name, db_names, gt_file)
    #     sum_ap += ap
    
    # # Compute mAP
    # m_ap = sum_ap / n_queries
    
    return total_results, m_ap

### $k$-d Trees
In this section you will use a set of randomized $k$-d trees to index the database of images. Write a function called `build_db_kdtrees` to build a set of randomized $k$-d trees given a set of descriptors:

> **Useful links**: [cv2.FlannBasedMatcher](https://docs.opencv.org/4.5.5/dc/de2/classcv_1_1FlannBasedMatcher.html), [Possible algorithms to create an index](https://docs.opencv.org/4.5.5/db/d18/classcv_1_1flann_1_1GenericIndex.html#a8fff14185f9f3d2f2311b528f65b146c), [Algorithms IDs](https://github.com/opencv/opencv/blob/master/modules/flann/include/opencv2/flann/defines.h#L70)

In [25]:
def build_db_kdtrees(descs, ntrees = 4):
    """
    Build a set of randomized k-d trees.
    
    - descs: A list of length len(img_names) where each element is a numpy array 
        of size (ndesc_for_this_image, 128). Each numpy array i corresponds 
        to the descriptors found on image i
    - ntrees: Number of trees to train
  
    RETURN: 
    - index: Trained FLANN index
    """  
  
     # Concatenate descriptors from all images
    descriptors = np.concatenate(descs)
    
    # Define FLANN parameters
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=ntrees)
    search_params = dict(checks=50)
    
    # Create FLANN index
    index = cv2.FlannBasedMatcher(index_params, search_params)
    index.add(descriptors)
    
    return index

In [26]:
# Simple example of DB construction
index = build_db_kdtrees(train_desc[0:2])
print(len(index.getTrainDescriptors()))

1103


In [27]:
# Search an image in the index
img_res = search_image(query_desc[0], index, train_names[0:2])
print(img_res)

AttributeError: ignored

In [ ]:
# Example of computing mAP
results, mAP = compute_mAP(query_names, query_desc, index, train_names[0:2])
print(results['100000.jpg'])
print(results['100100.jpg'])
print(mAP) # This should be 0 now, since there is only two images in the database.

**Q1**: Using functions developed so far, in the following cell compute the resulting **mAP** of the system **using 4 trees**:

In [ ]:
# Fill this variable with the resulting mAP
mAP_kdtree = 0.0

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
print('mAP: %.5f' % mAP_kdtree)

**Q2**: Are the results stable? Do you obtain always the same mAP? Why?

Write here the code required to answer the questions stated above. You can add more cells (code / markdown) at this point if you need it.

**Q3:** Analyze the effect of changing the number of trees in terms of mAP and average response time. Some plots here can be useful to justify your answer.

Write here the code required to answer the questions stated above. You can add more cells (code / markdown) at this point if you need it.

### Locality Sensitive Hashing (LSH)
In this section, you will use LSH to index the database of images. The LSH implementation included in OpenCV uses **bit sampling** for **Hamming distance** as a hash function and, therefore, binary descriptors must be used. Hence, SIFT descriptors are not valid and we need to describe the images, but using, for instance, ORB.

In the following cell, write the code required to generate **roughly 1500 keypoints / descriptors** using ORB for each query / train image:

> **Useful links**: [cv2.ORB_create](https://docs.opencv.org/4.5.4/db/d95/classcv_1_1ORB.html#aeff0cbe668659b7ca14bb85ff1c4073b)

In [ ]:
def generateAllDescriptorsAndKeypoints(query_images, train_images, no_keypoints=1500, no_descriptors=1500):
    """
    In this section, you will use LSH to index the database of images. The LSH implementation 
    included in OpenCV uses bit sampling for Hamming distance as a hash function and, therefore, 
    binary descriptors must be used. Hence, SIFT descriptors are not valid and we need to describe the images
    , but using, for instance, ORB.

    Useful links: cv2.ORB_create

    generate roughly 1500 keypoints / descriptors using ORB for each query / train image:
    
    - descs: A list of length len(img_names) where each element is a numpy array 
        of size (ndesc_for_this_image, 128). Each numpy array i corresponds 
        to the descriptors found on image i
    - ntrees: Number of trees to train

    RETURN: 
    - query_kps_orb: list of keypoints for query images
    - query_desc_orb: list of descriptors for query images
    - train_kps_orb: list of keypoints for train images
    - train_desc_orb: list of descriptors for train images
    """ 
    
    orb = cv2.ORB_create(nfeatures=no_keypoints, nlevels=8, scoreType=cv2.ORB_FAST_SCORE)

    # Initialize keypoint and descriptor lists for query and train images
    query_kps_orb = []
    query_desc_orb = []
    train_kps_orb = []
    train_desc_orb = []

    # Generate keypoints and descriptors for query images
    for query_image in query_images:
      img = cv2.cvtColor(query_image, cv2.COLOR_BGR2GRAY)
      kps, desc = orb.detectAndCompute(img, None)
      query_kps_orb.append(kps)
      query_desc_orb.append(desc)

    # Generate keypoints and descriptors for train images
    for train_image in train_images:
      img = cv2.cvtColor(train_image, cv2.COLOR_BGR2GRAY)
      kps, desc = orb.detectAndCompute(img, None)
      train_kps_orb.append(kps)
      train_desc_orb.append(desc)

    return query_kps_orb, query_desc_orb, train_kps_orb, train_desc_orb

In [ ]:
[query_kps_orb, query_desc_orb, train_kps_orb, train_desc_orb] = generateAllDescriptorsAndKeypoints(query_imgs, train_imgs)

In [ ]:
# Show some data
print(len(query_kps_orb[0]))
print(query_desc_orb[0].shape)
print(query_desc_orb[0])

1563
(1563, 32)
[[ 45  45  17 ... 251  82 160]
 [ 13 220 255 ... 146  84 171]
 [203   4 103 ... 244 112 116]
 ...
 [211  13 119 ... 160 165 116]
 [224  98 157 ... 240 138  11]
 [162  43 172 ...  93  90 207]]



Next, write a function called `build_db_lsh` to build a **standard** (*no multi-probe*) LSH index from a set of images:

In [ ]:
def build_db_lsh(descs, tables = 6, hash_size = 12):
    """
    build a standard (no multi-probe) LSH index from a set of images
    Index a set of images using LSH.    
    
    - descs: A list containing numpy arrays of size (~1500, 32). Each numpy array
        i corresponds to the ORB descriptors found at image i.
    - tables: Number of hash tables to create.
    - hash_size: Hash length in bits.
  
    RETURN: 
    - index: The trained LSH index.
    """  

ModuleNotFoundError: ignored

In [ ]:
print(train_desc_orb[0])

[[ 89  27 223 ... 224  33 113]
 [146 161 107 ... 130 171 169]
 [ 39  63 252 ... 223  71 245]
 ...
 [  3 141  39 ...   2 177 213]
 [ 41 203  17 ... 142 208 137]
 [116 106 233 ... 251  86 100]]


In [ ]:
# Simple example of DB construction
index = build_db_lsh(train_desc_orb[0:2])
print(len(index.getTrainDescriptors()))

AttributeError: ignored

**Q4**: In the following cell compute the resulting **mAP** of the system **using 6 tables and a hash size of 12**:

In [ ]:
# Fill this variable with the resulting mAP
mAP_lsh = 0.0

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
print('mAP: %.5f' % mAP_lsh)

**Q5**: Are the results stable? Do you obtain always the same mAP? Why?

Write here the code required to answer the questions stated above. You can add more cells (code / markdown) at this point if you need it.

**Q6**: Analyze the effect of changing the number of tables / hash size in terms of mAP and average response time. Some plots here can be useful to justify your answer.

Write here the code required to answer the questions stated above. You can add more cells (code / markdown) at this point if you need it.

**Q7:** Despite the different descriptors used, compare the performance of the randomized k-d trees and LSH approaches from different points of view (accuracy, training times, querying times, ...). Some plots can be useful here to justify your answer.

Write here the code required to answer the questions stated above. You can add more cells (code / markdown) at this point if you need it.

## Bag-of-Words
In this section, you will implement the Bag-of-Words (BoW) model for image retrieval. Additionally, you will also implement the TF-IDF scoring scheme.

### Download visual dictionaries
To use a BoW model, first we need a visual vocabulary. The authors of the INRIA Holidays dataset provide some visual vocabularies, trained using a clustering method (e.g. $k$-means) in a different dataset (Flickr60K).

First, download these vocabularies from [here](https://uibes-my.sharepoint.com/:u:/g/personal/egf350_id_uib_es/EY1G011OvfJOnwqWQQzmHmgBkXhLHBaK00wdizsUT252dw).

> **Unzip this file into the root directory of the development kit, at the same level of the datasets.**

A folder named `clust` is now available in your workspace, containing visual vocabularies of 100, 200, 500, 1K, 2K, 5K, 10K, 20K, 50K, 100K and 200K visual words. Again, these are binary files, and therefore we provide you with functions to load and index them:

In [34]:
%cd '../'

/content/drive/MyDrive/ImageIndexing


In [35]:
unzip_tarfile("clust.tar.gz")

In [29]:
voc = rd.load_visual_vocab("clust/clust_flickr60_k200.fvecs", ntrees=4)

With this function, the corresponding vocabulary is read. Additionally, a FLANN index structure based on kd-trees is built and returned using the centroids. This is to allow a fast access when searching for the closest visual words in the vocabulary. More precisely, in this example, 4 trees are constructed using the vocabulary of 200 centroids. Now, given a query descriptor(s), you can use `match` or `knnMatch` methods as usual to search for the closest (approximate) visual word(s) in the vocabulary.

### BoW and Inverted File
Now, write a class called `BoW` to manage the indexing procedure. This class should make use, in addition to the visual vocabulary, an inverted file to compute similarity scores between images. Apart from the class constructor, write three methods: `build_db`, `search_image` and `compute_mAP`:

In [30]:
class BoW:
    """
    Class to implement the BoW model + Inverted File.
    """
  
    def __init__(self, vocab_file):
        """
        Class constructor. It loads the vocabulary and initializes other stuff
        required for the CBIR system, such as the inverted file structure.
        """
        self.vocab = rd.load_visual_vocab(vocab_file)
        print(self.vocab)
        self.nwords = self.vocab.getTrainDescriptors()[0].shape[0]
        self.train_names = []
        self.inv_list = {word_id: {} for word_id in range(self.nwords)}

    def get_word_ids(self, descriptors):
      """
      Assign a visual word id to each descriptor.
      
      - descriptors: A numpy array of shape (ndesc, 128). Each row is a descriptor.
      
      RETURN:
      - word_ids: A numpy array of shape (ndesc,) containing the assigned visual word ids.
      """
      # Compute distances to the vocabulary words
      dists = cdist(self.vocab, descriptors, 'euclidean')
      
      # Assign the visual word id to each descriptor
      word_ids = np.argmin(dists, axis=0)
      
      return word_ids

    def build_db(self, img_names, img_descs):
        """
        Build an index from a set of images. Essentially, for each image, you should
        search its descriptors in the index in order to find the closest visual words
        and fill the inverted file structure consequently.
    
        - img_names: An ordered list with the names of the train images
        - img_descs: A list containing numpy arrays. Each numpy array i corresponds 
          to the descriptors found at image i
        """
        self.train_names = img_names
        for i, descriptors in enumerate(img_descs):
            # Assign a visual word id to each descriptor
            word_ids = self.get_word_ids(descriptors)
            
            # Update the inverted file structure
            for j, word_id in enumerate(word_ids):
                if i not in self.inv_list[word_id]:
                    self.inv_list[word_id][i] = []
                self.inv_list[word_id][i].append(j)

    def search_image(self, descs):
        """
        Search an image in the index.
      
        - descs: A numpy array. It is the set descriptors extracted from the query image.
    
        RETURN:
        - An ordered list of similar images, e.g.: ['100101.jpg', '100202.jpg', ...]
        """
        # query the LSH index to find the closest visual words
        word_ids = self.vocab.get_word_ids(self.lsh.query(descs))
        
        # get the image IDs associated with each visual word
        img_ids = set()
        for word_id in word_ids:
            img_ids.update(self.inv_file[word_id])
        
        # compute the image distances
        img_dist = []
        for img_id in img_ids:
            dist = np.linalg.norm(self.vocab.quantize(descs) - self.vocab.quantize(self.img_descs[img_id]))
            img_dist.append((self.img_paths[img_id], dist))
        
        # return the ordered list of similar images
        return [img_path for img_path, dist in sorted(img_dist, key=lambda x: x[1])]
        
    def compute_mAP(self, query_names, query_descs):
        """
        Perform a search for a list of query images against the database and evaluates
        the performance of the system.
        
        - query_names: An ordered list with the names of query images
        - query_descs: A list containing numpy arrays of size (ndesc_for_this_image, 128). 
              Each numpy array i corresponds to the descriptors found at image i.

        RETURN:
        - total_results: A dictionary containing, for each query image, an ordered list of the retrieved images.
        - m_ap: Mean Average Precision averaged over all queries.
        """
        total_results = {}
        m_ap = 0.0
        
        # search for each query image
        for i, (name, desc) in enumerate(zip(query_names, query_descs)):
            print("Querying image {} ({}/{})...".format(name, i+1, len(query_names)))
            results = self.search_image(desc)
            total_results[name] = results
            
            # compute average precision
            relevant = [int(n.split("_")[0] == name.split("_")[0]) for n in results]
            scores = np.arange(len(relevant), 0, -1)
            ap = average_precision_score(relevant, scores)
            m_ap += ap
        
        # average over all queries
        m_ap /= len(query_names)
        
        return total_results, m_ap

In [36]:
# Example of use
index = BoW('clust/clust_flickr60_k200.fvecs')
index.build_db(train_names[0:2], train_desc[0:2])

< cv2.FlannBasedMatcher 0x7f3236044850>


NameError: ignored

In [ ]:
res = index.search_image(query_desc[0])
print(res)

In [ ]:
results, mAP = index.compute_mAP(query_names, query_desc)
print(results)
print(mAP)

**Q8**: In the following cell compute the resulting mAP of the system **using the vocabularies of 200, 2K, 20K and 200K visual words**:

In [ ]:
# Fill these variables with the resulting mAP
mAP_200  = 0.0
mAP_2K   = 0.0
mAP_20K  = 0.0
mAP_200K = 0.0

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
print('mAP 200: %.5f' % mAP_200)
print('mAP 2K: %.5f' % mAP_2K)
print('mAP 20K: %.5f' % mAP_20K)
print('mAP 200K: %.5f' % mAP_200K)

**Q9**: Compare the performances obtained on each case. Is a larger vocabulary size always better? Why or why not?

Write here the code required to answer the questions stated above. You can add more cells (code / markdown) at this point if you need it.

**Q10**: Analyze the effect of the vocabulary size in terms of mAP and average response time (train and query times). Are these times constant for each vocabulary? Some plots here can be useful to justify your answer.

Write here the code required to answer the questions stated above. You can add more cells (code / markdown) at this point if you need it.

**Q11**: Do the results obtained depend on the set of images used to generate the vocabulary? How can we improve the retrieval performance?

Write here the code required to answer the questions stated above. You can add more cells (code / markdown) at this point if you need it.

### TF-IDF
As a final task of this assignment, let's implement the TF-IDF scoring scheme. Modify the `BoW` class you wrote before to include the TF-IDF weighting scheme:

In [38]:
class BoW_TFIDF:
    """
    Class to implement the BoW model + Inverted File + TF-IDF Scoring scheme
    """
  
    def __init__(self, vocab_file):
        """
        Class constructor. It loads the vocabulary and initializes other stuff
        required for the CBIR system, such as the inverted file structure.
        """
        self.vocab = rd.load_visual_vocab(vocab_file)
        self.nwords = self.vocab.getTrainDescriptors()[0].shape[0]
        self.train_names = []
        self.inv_list = {word_id: {} for word_id in range(self.nwords)}        
        self.tfidf = {}

    def build_db(self, img_names, img_descs):
        """
        Build an index from a set of images. Essentially, for each image, you should
        search its descriptors in the index in order to find the closest visual words
        and fill the inverted file structure consequently. Additionally, TF and IDF terms
        should be computed here.
    
        - img_names: An ordered list with the names of the train images
        - img_descs: A list containing numpy arrays. Each numpy array i corresponds 
          to the descriptors found at image i
        """
        self.inverted_files = {}
        for i, desc in enumerate(img_descs):
            words = self.quantize(desc)
            for w in set(words):
                if w not in self.inverted_files:
                    self.inverted_files[w] = []
                self.inverted_files[w].append(i)
        
        # compute IDF weights
        n_docs = len(img_descs)
        df = np.zeros(self.vocab_size, dtype=np.float32)
        for w, doc_ids in self.inverted_files.items():
            df[w] = len(doc_ids)
        idf = np.log((1 + n_docs) / (1 + df)) + 1
        self.idf_weights = TfidfTransformer(use_idf=True, smooth_idf=True, sublinear_tf=False)
        self.idf_weights.fit(idf.reshape(1, -1))

    def search_image(self, descs):
        """
        Search an image in the index. Use the TF-IDF here when scoring the images.
      
        - descs: A numpy array. It is the set descriptors extracted from the query image.
    
        RETURN:
        - An ordered list of similar images, e.g.: ['100101.jpg', '100202.jpg', ...]
        """
        query_words = self.quantize(descs)
        query_tf = np.bincount(query_words, minlength=self.vocab_size)
        query_tf_idf = self.idf_weights.transform(query_tf.reshape(1, -1)).toarray()[0]
        
        # compute cosine similarity between query and documents
        scores = []
        for i, desc in enumerate(self.visual_words):
            doc_tf_idf = self.idf_weights.transform(desc.reshape(1, -1)).toarray()[0]
            sim = np.dot(query_tf_idf, doc_tf_idf) / np.linalg.norm(query_tf_idf) / np.linalg.norm(doc_tf_idf)
            scores.append((i, sim))
        scores.sort(key=lambda x: x[1], reverse=True)
        
        # return list of document names sorted by similarity
        results = [img_names[i] for i, _ in scores]
        return results
        
    def compute_mAP(self, query_names, query_descs):
        """
        Perform a search for a list of query images against the database and evaluates
        the performance of the system.
        
        - query_names: An ordered list with the names of query images
        - query_descs: A list containing numpy arrays of size (ndesc_for_this_image, 128). 
              Each numpy array i corresponds to the descriptors found at image i.

        RETURN:
        - total_results: A dictionary containing, for each query image, an ordered list of the retrieved images.
        - m_ap: Mean Average Precision averaged over all queries.
        """
        total_results = {}
        m_ap = 0.0
        
        # search for each query image
        for i, (name, desc) in enumerate(zip(query_names, query_descs)):
            print("Querying image {} ({}/{})...".format(name, i+1, len(query_names)))
            results = self.search_image(desc)
            total_results[name] = results
            
            # compute average precision
            relevant = [int(n.split("_")[0] == name.split("_")[0]) for n in results]
            scores = np.arange(len(relevant), 0, -1)
            ap = average_precision_score(relevant, scores)
            m_ap += ap
        
        # average over all queries
        m_ap /= len(query_names)
        
        return total_results, m_ap

In [39]:
# Example of use
index = BoW_TFIDF('clust/clust_flickr60_k200.fvecs')
index.build_db(train_names[0:2], train_desc[0:2])

AttributeError: ignored

In [ ]:
res = index.search_image(query_desc[0])
print(res)

**Q12**: In the following cell compute the resulting mAP of the system **using the vocabularies of 200, 2K, 20K and 200K visual words**:

In [ ]:
# Fill these variables with the resulting mAP
mAP_200  = 0.0
mAP_2K   = 0.0
mAP_20K  = 0.0
mAP_200K = 0.0

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
print('mAP 200: %.5f' % mAP_200)
print('mAP 2K: %.5f' % mAP_2K)
print('mAP 20K: %.5f' % mAP_20K)
print('mAP 200K: %.5f' % mAP_200K)

**Q13:** Compare the performances obtained on each case. Is a larger vocabulary size always better? Why or why not?

Write here the code required to answer the questions stated above. You can add more cells (code / markdown) at this point if you need it.

**Q14**: Analyze the effect of the vocabulary size in terms of mAP and average response time (train and query times). Are these times constant for each vocabulary? Some plots here can be useful to justify your answer.

Write here the code required to answer the questions stated above. You can add more cells (code / markdown) at this point if you need it.

**Q15**: Do the results obtained depend on the set of images used to generate the vocabulary? How can we improve the retrieval performance?

Write here the code required to answer the questions stated above. You can add more cells (code / markdown) at this point if you need it.

**Q16:** How does TF-IDF affect the performance? Better or worse? Does this make sense?

Write here the code required to answer the questions stated above. You can add more cells (code / markdown) at this point if you need it.

## Submitting your work

**Important**: Please make sure that the submitted notebooks have been run and the cell outputs are visible.

**Important**: Please make also sure that you have filled the **NAME** and **DNI** variables at the beginning of the notebook, **using the indicated format**.

Once you have filled out the necessary code and you are happy with your solution, **save your notebook** and execute the following cell:

In [ ]:
zip_filename = DNI + '_A2.zip'
zf = zipfile.ZipFile(zip_filename, mode = 'w')

aname = 'submitted/' + DNI + '/A2/Image_Indexing.ipynb'
zf.write('Image_Indexing.ipynb', arcname = aname);

zf.close()

This will generate a zip file of your code called `DNI_A2.zip` in the same directory of the assignment. This is the file that you must upload to [Aula Digital](https://uibdigital.uib.es/) to submit your work!

---

&copy; Emilio Garcia-Fidalgo, University of the Balearic Islands